In [2]:
import pandas as pd
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from database_model import (
    Base, Country, Measure, MeasureDate, MeasureDetail,
    MeasureModification, MeasurePolicyLink, PolicyMeasureLevel
)

# Parametry połączenia
user = 'postgres'
password = 'postgres'
host = 'localhost'
port = '5433'
database = 'covid_fci'
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

# Wczytanie danych
df = pd.read_csv('sources/covid-fci-data-cleaned.csv', encoding='utf-8', parse_dates=["Date", "Termination Date"])
df = df.where(pd.notnull(df), None)

# Rozpoczęcie sesji
session = Session(engine)

for _, row in df.iterrows():
    # Kraje
    country = session.query(Country).filter_by(iso3=row['Country ISO3']).first()
    if not country:
        country = Country(
            iso3=row['Country ISO3'],
            name=row['Country Name'],
            income_level=row['Income Level']
        )
        session.add(country)
        session.flush()

    # Measures
    measure = Measure(original_id=row['Original_ID'], country=country)
    session.add(measure)
    session.flush()

    # MeasureDate (konwersja NaT na None)
    date = row['Date'] if pd.notna(row['Date']) else None
    termination_date = row['Termination Date'] if pd.notna(row['Termination Date']) else None
    date_obj = MeasureDate(measure_id=measure.id, date=date, termination_date=termination_date)
    session.add(date_obj)

    # MeasureDetail
    detail = MeasureDetail(
        measure_id=measure.id,
        authority=row['Authority'],
        details=row['Details of the measure'],
        reference=row['Reference']
    )
    session.add(detail)

    # MeasureModification
    was_modified = str(row['Modification of Parent Measure']).lower() != 'no' if row['Modification of Parent Measure'] else False
    mod = MeasureModification(
        measure_id=measure.id,
        was_modified=str(was_modified),
        modification_of_parent=row['Modification of Parent Measure'],
        parent_measure=row['Parent Measure']
    )
    session.add(mod)

    # PolicyMeasureLevels + Link
    for col, level_type in zip(
        ['Level 1 policy measures', 'Level 2 policy measures', 'Level 3 policy measures'],
        ['level1', 'level2', 'level3']
    ):
        if row[col]:
            policy = session.query(PolicyMeasureLevel).filter_by(name=row[col], level_type=level_type).first()
            if not policy:
                policy = PolicyMeasureLevel(name=row[col], level_type=level_type)
                session.add(policy)
                session.flush()

            link = MeasurePolicyLink(measure_id=measure.id, policy_measure_level_id=policy.id)
            session.add(link)

# Zatwierdzenie
session.commit()
session.close()
